### Pytorch EfficientNet Feature Extraction
This notebook demonstrate the usage of extracting features from the CT scan images by using EfficientNet(Pytorch implementation).

If you'd like to add the information of CT scan images into your Feature-Engineering process, you can follow the methodology of this notebook to obtain features from each patient’s images.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import cv2
import pydicom

In [ ]:
! pip install efficientnet_pytorch

In [ ]:
def get_img(path):
    d = pydicom.dcmread(path)
    # normlize and resize the image
    return cv2.resize((d.pixel_array - d.pixel_array.min()) / (d.pixel_array.max() - d.pixel_array.min()), (512, 512))

In [ ]:
ROOT = "../input/osic-pulmonary-fibrosis-progression"
os.listdir(ROOT)

In [ ]:
# load a demo image
demo_img_path = os.path.join(ROOT, "train", "ID00007637202177411956430", "1.dcm")
demo_img = get_img(demo_img_path)
demo_img.shape

In [ ]:
# get the efficientNet model
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b6')
model.cuda()

In [ ]:
import torch
# convert the image to the input tensor, the input must be a 3-channel tensor with float dtype
demo_img_ts = torch.from_numpy(demo_img).unsqueeze(0).unsqueeze(0).repeat(1, 3, 1, 1).to(dtype=torch.float).cuda()
demo_img_ts.size()

As you can see, when the input image is in shape (batch, 3, 512, 512). The feature tensor extracted by the efficientNet-b6 is in shape (batch, 2304, 16, 16). You can flatten this tensor or send it through a few more convolution layers to make it adapt to your model's input size.
### NOTE
After extraction, you might need to detach this tensor from the graph then send it into your model for fvc regression. Otherwise, you might fine-tune the weights of the efficientNet when you train your fvc regression model.

In [ ]:
demo_feature = model.extract_features(demo_img_ts)
demo_feature.size()